# Coding-Test
* Candidate: José Maria Clementino Junior

------------------------------------------------

Necessary imports

In [1]:
import pandas as pd
import os 
import glob
! pip install pandasql
from pandasql import sqldf
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
current_directory = os.getcwd()
mydir = current_directory
file_list = glob.glob(mydir + "/*.csv")

In [3]:
file_list

['C:\\Users\\junin\\Downloads\\codingSkills-main\\codingSkills-main\\admissions.csv',
 'C:\\Users\\junin\\Downloads\\codingSkills-main\\codingSkills-main\\d_labitems.csv',
 'C:\\Users\\junin\\Downloads\\codingSkills-main\\codingSkills-main\\edstays.csv',
 'C:\\Users\\junin\\Downloads\\codingSkills-main\\codingSkills-main\\labevents.csv']

In [4]:
list_df = []
for i in file_list:
    df = pd.read_csv(i)
    list_df.append(df)

In [5]:
df_admissions = list_df[0]
df_labitems = list_df[1]
df_edstays = list_df[2]
df_labevents = list_df[3]

## Reading the data

In [6]:
df_admissions.head(5)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24T14:38:00,2196-03-04T14:02:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2196-02-24T12:15:00,2196-02-24T17:07:00,0
1,10009628,25926192,2153-09-17T17:08:00,2153-09-25T13:20:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO,NaN,NaN,0
2,10018081,23983182,2134-08-18T02:02:00,2134-08-23T19:35:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17T16:24:00,2134-08-18T03:15:00,0
3,10006053,22942076,2111-11-13T23:39:00,2111-11-15T17:20:00,2111-11-15T17:20:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04T18:46:00,2113-08-06T20:57:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0


In [7]:
df_edstays.head(5)

,subject_id,hadm_id,stay_id,intime,outtime
0,10014729,23300884.0,32435236,2125-03-19T12:36:00,2125-03-19T16:59:47
1,10004235,24181354.0,36583389,2196-02-24T12:15:00,2196-02-24T17:07:00
2,10026255,22059910.0,30926886,2201-07-07T12:31:00,2201-07-07T19:40:00
3,10026255,20437651.0,36948716,2200-09-17T18:38:00,2200-09-18T00:57:00
4,10007058,22954658.0,35770266,2167-11-07T17:57:00,2167-11-07T20:22:00


In [8]:
df_labevents.head(5)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,183881,10014354,29600294.0,1808066,51277,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.4,%,10.5,15.5,NaN,ROUTINE,NaN
1,183883,10014354,29600294.0,1808066,51301,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.3,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
2,183888,10014354,29600294.0,1808066,52167,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.7,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,183870,10014354,29600294.0,1808066,51249,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.1,g/dL,32.0,37.0,abnormal,ROUTINE,NaN
4,183865,10014354,29600294.0,1808066,51222,2148-08-16 00:00:00,2148-08-16 01:30:00,9.2,9.2,g/dL,13.7,17.5,abnormal,ROUTINE,NaN


In [9]:
df_labitems.head(5)

,itemid,label,fluid,category,loinc_code
0,52016,Abz,Blood,Blood Gas,NaN
1,52017,"Albumin, Blood",Blood,Blood Gas,NaN
2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,NaN
3,52018,Assist/Control,Blood,Blood Gas,NaN
4,50802,Base Excess,Blood,Blood Gas,NaN
